In [14]:
# -*- coding: utf-8 -*-

from elasticsearch6 import Elasticsearch
from elasticsearch6.exceptions import TransportError
from elasticsearch6.helpers import bulk, streaming_bulk
import csv
import json

#2016~2020

inputs = open('setting.json', 'r', encoding='utf-8')
setting = json.load(inputs)['setting']

keyword = "정세균"
start_date = "2020-01-01"
end_date = "2020-12-31"

es_client = Elasticsearch(hosts=[{'host': setting['db']['host'], 'port': setting['db']['port'], 'requestTimeout': 60*10, 'keepAlive': False, 'log': 'debug'}],timeout= 60*10, max_retries= 10, retry_on_timeout= True)

query = {
    "_source": ["reg_date","body","html","username","length","replies","likes","is_video","aid","keyword"],
    "size": 1000,
    "from": 0,
    "query":{
        "bool": {
            "must":[
                {"term": {"type.keyword": "sns"}},
                {"term": {"origin_nm": "instagram"}},
                {"term": {"keyword": keyword}}
            ],
        "filter":{
            "range": {
                "reg_date": {
                    "gte": start_date,
                    "lte": end_date,
                    "format" : "yyyy-MM-dd"
                    }
                }
            }
        }
    }
}

res = es_client.search(body=query, index='news', timeout='10m')

bodys = []

for data in res['hits']['hits']:
    bodys.append(data['_source']['body'])

with open("2020_정세균.json", "w") as json_file:
    json.dump(bodys, json_file)

    
    